In [33]:
%pip install guidance
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import guidance
from dotenv import load_dotenv

load_dotenv()

# To get a different result every time, uncomment the following line to clear the cache
# guidance.llms.OpenAI.cache.clear()

guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

#### Guardrail the LLM output structure

In [35]:
program = guidance('''
This is a rhyme by {{author}}:

"Roses are red, the water is blue {{gen "rhyme"}}

Another rhyme by {{author}}:
"{{gen "rhyme"}}"
''')

quote = program(author="janik")

quote


Stop program 
This is a rhyme by janik :

"Roses are red, the water is blue 
The sky is so high, and so are you 
Your beauty is like a summer day 
Your smile is like a ray of light that lights my way" 

Another rhyme by janik :
" The sun is so bright, the stars so high 
Your love is like a sweet lullaby 
Your eyes sparkle like the morning dew 
Your heart is so pure, it's all I need to get me through" "

#### Restrict LLM output selection

In [36]:
options = ['Yes', 'No', 'Maybe']

program = guidance('''Is the following sentence offensive? 
Sentence: {{example}}
Answer:{{select "answer" options=options}}''')
                   
executed_program = program(example='your tacos taste can be improved', options=options)


Stop program Is the following sentence offensive? 
Sentence: your tacos taste can be improved 
Answer: No

#### If conditions & Hidden generation

In [37]:
options = ["yes", "no"]

program = guidance(
'''User: {{query}}

{{#block hidden=True~}}
Is the user query rude or offensive?
Answer: {{select "rude" options=options}}
{{~/block~}}

{{#if rude=="yes"}}Assistant: Please be polite{{~/if~}}
{{#if rude=="no"}}Assistant: {{gen "answer"}}{{~/if~}}
''')
executed_program = program(query='your tacos taste too salty', options=options)



Stop program User: your tacos taste too salty 

 Assistant: I'm sorry to hear that. Is there anything else I can do to help?

# Generate emails with if condition

In [38]:
priorities = ["low priority", "medium priority", "high priority"]

email = guidance('''    
{{#block hidden=True~}}
Here is the customer message we received: {{email}}
Please give it a priority score
priority score: {{select "priority" options=priorities}}
{{~/block~}}
        
{{#block hidden=True~}}
You are a world class customer support; Your goal is to generate an response based on the customer message;
Here is the customer message to respond: {{email}}
Generate an opening & one paragraph of response to the customer message based on priority score: {{priority}}:
{{gen 'email_response'}} 
{{~/block~}}

{{email_response}}
{{#if priority=='high priority'}}Would love to setup a call this/next week, here is the calendly link: https://calendly.com/janik-dotzel{{/if}}
Best regards
Janik
''')

email_response = email(email='What features does webflow have but wix dont have? I need an urgent answer', priorities=priorities)


Stop program 
 
Thanks for reaching out with your question! Webflow offers a range of features that Wix does not, such as the ability to create custom code, access to a wide range of third-party integrations, and the ability to create dynamic content. Additionally, Webflow also offers a powerful visual editor, allowing you to create complex designs without needing to write code. I hope this helps answer your question! 
 Would love to setup a call this/next week, here is the calendly link: https://calendly.com/janik-dotzel 
Best regards
Janik

# Generate chart

In [39]:
import urllib.parse

def generate_chart(query):
    
    def parse_chart_link(chart_details):
        encoded_chart_details = urllib.parse.quote(chart_details, safe='')

        output = "![](https://quickchart.io/chart?c=" + encoded_chart_details + ")"

        return output
    
    examples = [
        {
            'input': "Make a chart of the 5 tallest mountains",
            'output': {"type":"bar","data":{"labels":["Mount Everest","K2","Kangchenjunga","Lhotse","Makalu"], "datasets":[{"label":"Height (m)","data":[8848,8611,8586,8516,8485]}]}}
        },
        {
            'input': "Create a pie chart showing the population of the world by continent",
            'output': {"type":"pie","data":{"labels":["Africa","Asia","Europe","North America","South America","Oceania"], "datasets":[{"label":"Population (millions)","data": [1235.5,4436.6,738.8,571.4,422.5,41.3]}]}}
        }
    ]

    guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

    chart = guidance(
    '''    
    {{#block hidden=True~}}
    You are a world class data analyst, You will generate chart output based on a natural language;

    {{~#each examples}}
    Q:{{this.input}}
    A:{{this.output}}
    ---
    {{~/each}}
    Q:{{query}}
    A:{{gen 'chart' temperature=0 max_tokens=500}}    
    {{/block~}}
    
    Hello here is the chart you want
    {{parse_chart_link chart}}
    ''')

    return chart(query=query, examples=examples, parse_chart_link=parse_chart_link)


In [40]:
player_and_points = {
    'Felix Biller': 1687,
    'Finn Braunz': 1660,
    'Janik Dotzel': 1856,
    'Hans Michael Dudek': 1755,
    'David Eckl': 1786,
    'Christian Fredrich': 1824,
    'Luis Freundschig': 1704,
    'Nico Gremminger': 1761,
    'Bastian Herbert': 1983,
    'Paul Hofmann': 1877,
    'Maximilian Interwies': 1682,
    'Luis Kasten': 1922,
    'Lenny Kreis': 1781,
    'Felix Lenz': 1755,
    'Kamil Michalik': 2129,
    'Stanislaw Miller': 1782,
    'Max Nötzold': 1948,
    'Nico Pfrenzinger': 1661,
    'Lennart Rieger': 1797,
    'Christian Röder': 1786,
    'Christoph Sasse': 1945,
    'Jonas Scheer': 1867,
    'Michael Schüller': 1727,
    'Adrian Späte': 1669,
    'Janik Welsch': 1713,
    'Vanessa Welsch': 1532,
    'Andreas von Hermanni': 1406
}

chart = generate_chart('''
Create a pie chart showing how many players score group. 
A score group is every 100 points. E.g. 1700-1799 is one score group.
You don't need to show groups that have no players.
Make sure that players with a score higher thatn 2000 are in the chart.
Double check that the chart is correct. Correctness is the highest priority.
''' + str(player_and_points))

chart

Stop program 
 Hello here is the chart you want
 ![](https://quickchart.io/chart?c=%20%7B%27type%27%3A%20%27pie%27%2C%20%27data%27%3A%20%7B%27labels%27%3A%20%5B%271600-1699%27%2C%20%271700-1799%27%2C%20%271800-1899%27%2C%20%271900-1999%27%2C%20%272000%2B%27%5D%2C%20%27datasets%27%3A%20%5B%7B%27label%27%3A%20%27Number%20of%20Players%27%2C%20%27data%27%3A%20%5B7%2C%2011%2C%206%2C%205%2C%201%5D%7D%5D%7D%7D)

# Embed text to image 

In [41]:
def generate_story(story_idea):
        
    story = guidance('''
{{#block hidden=True~}}
You are a world class story teller; Your goal is to generate a short tiny story less than 200 words based on a story idea;

Here is the story idea: {{story_idea}}
Story: {{gen 'story' temperature=0}}
{{/block~}}

{{#block hidden=True~}}
You are a world class AI artist who is great at generating text to image prompts for the story; 
Your goal is to generate a good text to image prompt and put it in a url that can generate image from the prompt;

Story: You find yourself standing on the deck of a pirate ship in the middle of the ocean. You are checking if there are still people on the ship.
Image url markdown: ![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20man%20standing%20on%20the%20deck%20of%20a%20pirate%20ship%20in%20the%20middle%20of%20the%20ocean)
                
Story: {{story}}
Image url markdown: {{gen 'image_url' temperature=0 max_tokens=500}})
{{~/block~}}
                     
Story: {{~story~}}
{{image_url}}
''')

    story = story(story_idea=story_idea)
    print(story)
    return story

In [42]:
generate_story("little boy dreaming")

Stop program 
Story: 

The little boy lay in his bed, dreaming of a world of adventure. He imagined himself soaring through the sky, soaring higher and higher until he could see the entire world below him. He imagined himself exploring the depths of the ocean, discovering new creatures and uncovering hidden treasures. He imagined himself climbing the highest mountains, conquering the toughest obstacles and reaching the peak of success.

The little boy's dreams were filled with excitement and possibility. He dreamed of a life of exploration and discovery, of pushing himself to the limits and achieving greatness. He dreamed of a life of adventure and of making a difference in the world.

The little boy's dreams were his escape from the mundane reality of his everyday life. He dreamed of a life of freedom and of living life to the fullest. He dreamed of a life of joy and of making a lasting impact on the world.

The little boy's dreams were his own, and he cherished them deeply. He knew that one day, he would make them come true. 
![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20little%20boy%20lying%20in%20bed%20dreaming%20of%20a%20world%20of%20adventure)